In [6]:
import requests
import pandas as pd
import datetime

url = f"https://production.dataviz.cnn.io/index/fearandgreed/graphdata/2022-01-01"

session = requests.Session()

# Optional: set headers (some APIs require a User-Agent)
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/117.0 Safari/537.36"
})

response = session.get(url)
response.raise_for_status()
r = response.json()

data = r["fear_and_greed_historical"]["data"]
df = pd.DataFrame(data)
df["x"] = pd.to_datetime(df["x"], unit="ms")  # convert timestamp
df = df.rename(columns={"x":"date", "y":"fear_greed_index"})


In [15]:
df

,date,fear_greed_index,rating
0,2022-01-03 00:00:00,65.228571,greed
1,2022-01-04 00:00:00,64.257143,greed
2,2022-01-05 00:00:00,51.028571,neutral
3,2022-01-06 00:00:00,52.142857,neutral
4,2022-01-07 00:00:00,49.257143,neutral
...,...,...,...
932,2025-09-22 00:00:00,62.285714,greed
933,2025-09-23 00:00:00,58.457143,greed
934,2025-09-24 00:00:00,56.428571,greed
935,2025-09-25 00:00:00,56.428571,greed


In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.types import VARCHAR, INTEGER, FLOAT, DateTime




df['weekly_fear_greed_index_max'] = (
    df.groupby(pd.Grouper(key='date', freq='W-SUN'))['fear_greed_index']
      .transform('max')
)

df['weekly_fear_greed_index_min'] = (
    df.groupby(pd.Grouper(key='date', freq='W-SUN'))['fear_greed_index']
      .transform('min')
)

# with engine.begin() as conn:
#     conn.execute(text('TRUNCATE TABLE public."users" RESTART IDENTITY CASCADE;'))
#     # conn.execute(text('drop table public.fear_greed_index;'))

# 최초 생성 시 dtype 명시 가능
df.head(0).to_sql("fear_greed_index", engine, schema="public", if_exists="replace", index=False, dtype={"date": DateTime, "fear_greed_index": FLOAT, "rating": VARCHAR(50)})
df.to_sql("fear_greed_index", engine, schema="public", if_exists="append", index=False, method="multi", chunksize=10_000)


937

In [25]:
week_key = df['date'].dt.to_period('W-SUN')  # 각 행이 속한 주(월~일)
week_key


0      2022-01-03/2022-01-09
1      2022-01-03/2022-01-09
2      2022-01-03/2022-01-09
3      2022-01-03/2022-01-09
4      2022-01-03/2022-01-09
               ...          
932    2025-09-22/2025-09-28
933    2025-09-22/2025-09-28
934    2025-09-22/2025-09-28
935    2025-09-22/2025-09-28
936    2025-09-22/2025-09-28
Name: date, Length: 937, dtype: period[W-SUN]

,date,fear_greed_index,rating,weekly_fear_greed_index
0,2022-01-03 00:00:00,65.228571,greed,49.257143
1,2022-01-04 00:00:00,64.257143,greed,49.257143
2,2022-01-05 00:00:00,51.028571,neutral,49.257143
3,2022-01-06 00:00:00,52.142857,neutral,49.257143
4,2022-01-07 00:00:00,49.257143,neutral,49.257143
...,...,...,...,...
932,2025-09-22 00:00:00,62.285714,greed,56.428571
933,2025-09-23 00:00:00,58.457143,greed,56.428571
934,2025-09-24 00:00:00,56.428571,greed,56.428571
935,2025-09-25 00:00:00,56.428571,greed,56.428571
